## Final Project Submission

Please fill out:
* __Student name__: Hannah Kwirikia
* __Student pace__: full time
* __Scheduled project review date/time__: 20th Nov.2022/ 11:59PM
* __Instructor name__: Mark Tiba 
* __Blog post URL__:


# Microsoft's Movie Market Analysis - What type of films should they create?

## Research Objectives

To find out what types of films are currently doing the best at the box office.

In [1]:
pwd

'C:\\Users\\Win 10 Pro\\Desktop\\dsc-phase-1-project-v2-4'

In [46]:
cd C:\\Users\\Win 10 Pro\\Desktop\\dsc-phase-1-project-v2-4\\zippedData

C:\Users\Win 10 Pro\Desktop\dsc-phase-1-project-v2-4\zippedData


## Import Libraries

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
%matplotlib inline

### Unzip SQL database file

In [50]:
#to unzip IMDB file
import zipfile
with zipfile.ZipFile('im.db.zip', 'r') as my_zip:
    my_zip.extractall('files') # the unzipped files are now in a folder named files

In [49]:
#to view list of tables
conn = sqlite3.connect('im.db')
cur = conn.cursor()
cur.execute("""SELECT name FROM sqlite_master WHERE type = 'table';""")
table_names = cur.fetchall()
table_names

[]

The tables for the 'movie_basics' and 'movie_ratings' are the most important. We will look at them below.

In [43]:
#to view the movie_basics table
movie_basics = pd.read_sql("""SELECT * FROM movie_basics;""", conn )
movie_basics

DatabaseError: Execution failed on sql 'SELECT * FROM movie_basics;': no such table: movie_basics